In [1]:
import cudaq
import numpy as np
import time
from cudaq import spin
import scipy

#cudaq.set_target('tensornet')

@cudaq.kernel
def two_qubit_gate(angle:float, qubit_1: cudaq.qubit, qubit_2: cudaq.qubit):  # mode: str = "CNOT_decomposition"  [cudaq doesn't support string type]
    x.ctrl(qubit_1, qubit_2)
    rz(angle, qubit_2)
    x.ctrl(qubit_1, qubit_2)

@cudaq.kernel
def Trotter_circuit(N: int, k:int, angles_ry:np.ndarray, angles_u3:np.ndarray, angles_2q:np.ndarray):  #list[int]
    # This is the actual Trotter circuit. Here the circuit construction for Trotterized version of time evolution happens
    # k : Trotter repeat length

    qreg=cudaq.qvector(N)

    for i in range(N):
        ry(angles_ry[i], qreg[i])

    for _ in range(k-1):
        for i in range(N):
            u3(angles_u3[i*3], angles_u3[i*3+1], angles_u3[i*3+2], qreg[i])

        for i in range(N):
            for j in range(i + 1, N): 
                two_qubit_gate(angles_2q[i*N+j], qreg[i], qreg[j])

    for i in range(N):
        u3(angles_u3[i*3], angles_u3[i*3+1], angles_u3[i*3+2], qreg[i])


In [2]:
def Euler_angle_decomposition(U:np.ndarray):
    theta = 2 * np.arccos(np.abs(U[0, 0]))

    phi = np.angle(U[1, 0]) - np.angle(U[0, 0])
    lam = np.angle(U[1, 1]) - np.angle(U[1, 0])
    
    return theta, phi, lam

In [3]:
def computing_norm_ratio(N,l,J):  #This gives value of alpha = self.computing_norm_ratio()
    #This computes alpha = ||H_x||_F/||H_z||_F using params only. No storing and computing full matrix is necessary
    #Coupling_matrix = np.reshape(np.array(self.model_instance_two_body), (self.no_spins, self.no_spins), order='F')
    alpha = np.sqrt(N)/np.sqrt(sum([J**2 for J in J[np.tril_indices(N, k = -1)]]) + sum([h**2 for h in l]))
    return alpha

def get_spinconfig_to_int(spinconfig:np.ndarray):
    #The inverse of int_to_spinconfig. Example: if spinconfig is int(np.array([-1,-1,1])) then output is 1. Given a config it first
    # undo the array as -1 is changed to 0 and 1 is left alone.
    bit_string = ''.join(['0' if spin==-1 else '1' for spin in spinconfig])

    return int(bit_string, 2)  

def calculate_angles(N,l,J):
    alpha = computing_norm_ratio(N,l,J)
    #time_array, time_delta_step = self.scalar_time_sampling(sampling_type="discrete")
    time_delta = 0.5
    #gamma_array, gamma_step = self.scalar_gamma_sampling(sampling_type="discrete")
    gamma = 0.42

    angles_u3 = np.zeros(3*N)
    for qubit in range(N):
        coeff = -alpha*(1-gamma)*l[N-1-qubit]
        one_body_Ham = gamma * spin.x(0) + coeff * spin.z(0)
        angles_u3[3*qubit: 3*(qubit+1)] = list(Euler_angle_decomposition(scipy.linalg.expm(-1.0j*time_delta*one_body_Ham.to_matrix())))   # always 2*2 so no problem of exponentiation, storage

    angles_2q = np.zeros((N,N))
    for i in range(N):
        for j in range(i+1, N):
            angles_2q[i,j] = 2*J[N-1-i, N-1-j]*(1-gamma)*alpha*time_delta

    return angles_u3, angles_2q

In [ ]:
def dict_to_res(counts):
  for key, value in counts.items():
    if value == 1: 
        final_config = key

  res = [1.0 if s == '1' else -1.0 for s in final_config]
  
  return np.array(res)


def Sampling(N,sample_size, burn, init_config=[]):
  k = 24
  l = np.random.randn(N)   #one body coefficients
  J = np.random.randn(N,N)   #two body coefficients

  tim = time.time()
  prob_dict = {}
  sample_list = []

  if len(init_config)==0:
    s = np.random.choice([1.,-1.],size=N)
  else: s = init_config

  angles_u3, angles_2q = calculate_angles(N,l,J)

  for _ in range(burn):
    angles_ry = np.pi*(s + 1)/2
    counts = cudaq.sample(Trotter_circuit, N, k, angles_ry, angles_u3, np.reshape(angles_2q,-1), shots_count=1)
    #counts = Trotter_circuit_builder(N, k, angles_ry, angles_u3, np.reshape(angles_2q,-1), shots_count=1)
    s = dict_to_res(counts)
  
  print("Burn Complete!")

  for _ in range(sample_size):
    angles_ry = np.pi*(s + 1)/2
    counts = cudaq.sample(Trotter_circuit, N, k, angles_ry, angles_u3, np.reshape(angles_2q,-1), shots_count=1)
    #counts = Trotter_circuit_builder(N, k, angles_ry, angles_u3, np.reshape(angles_2q,-1), shots_count=1)
    s = dict_to_res(counts)
    
    key = get_spinconfig_to_int(s)
    if key in prob_dict: prob_dict[key] +=1
    else: prob_dict[key] = 1
    sample_list.append(s)

  print("Sampling Time: ", time.time()-tim)
  
  prob_dict_flipped = {}
  for key in prob_dict.keys():
    prob_dict_flipped[2**N - key - 1] = prob_dict[key] / sample_size

  return np.flip(prob_dict_flipped), sample_list     #This flip is required to make peace with the difference in the convention.


def main(N, sample_size, dir):
    prob_dict, sample_list = Sampling(N, sample_size, burn=sample_size//10)

    np.save(dir+'TFIM_samples_mode=kernel_N='+str(N)+'.npy', sample_list)

    import pickle
    with open(dir+'TFIM_prob_dict_N='+str(N)+'.pkl', 'wb') as f:
        pickle.dump(prob_dict, f)


In [5]:
main(4,100,1,'Data/')

Burn Complete!
Sampling Time:  2.379516363143921


In [6]:
import argparse
if __name__ == "__main__":

    Trotter_circuit.compile()
    
    parser = argparse.ArgumentParser()
    parser.add_argument('N', type=int, help='The system size')
    parser.add_argument('--sample_size', type=int, default=100)
    parser.add_argument('--seed', type=int, default=1)
    parser.add_argument('--dir', type=str, default='Data/')

    args = parser.parse_args()
    tm=time.time()
    main(args.N, args.sample_size, args.seed, args.dir)
    print("Total Time: ", time.time()-tm)

usage: ipykernel_launcher.py [-h] [--sample_size SAMPLE_SIZE] [--seed SEED]
                             [--dir DIR]
                             N
ipykernel_launcher.py: error: the following arguments are required: N


SystemExit: 2

/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3587: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
